<a href="https://colab.research.google.com/github/colesmcintosh/pycuda-numpy-vector-ops/blob/main/Accelerating_NumPy_Vector_Operations_with_PyCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Accelerating NumPy Vector Operations with PyCUDA

This notebook demonstrates how to accelerate large-scale NumPy operations using GPU programming in Python via [PyCUDA](https://documen.tician.de/pycuda/).

We compare traditional CPU-based NumPy operations with a GPU-accelerated fused multiply-add (FMA) operation:

> The operation is defined as $c[i] = a[i] \times b[i] + d[i]$.

The notebook uses:
- Pinned (page-locked) memory for faster host-device transfers
- CUDA streams for asynchronous execution
- Event timing for accurate benchmarks

The result is a fast, validated comparison of NumPy vs PyCUDA performance.

In [1]:
!pip install pycuda --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.5 MB/s eta 0:00:00


## CUDA Kernel: Fused Multiply-Add
We use a fused multiply-add operation

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import time

kernel_code = """
__global__ void fused_op(float *a, float *b, float *d, float *c, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        c[idx] = a[idx] * b[idx] + d[idx];
    }
}
"""

mod = SourceModule(kernel_code)
fused_op = mod.get_function("fused_op")

## Set Array Size and Grid Configuration

In [3]:
N = 10_000_000
threads_per_block = 256
blocks_per_grid = (N + threads_per_block - 1) // threads_per_block

## Allocate GPU Buffers and Structured Pinned Memory

In [4]:
a = cuda.pagelocked_empty(N, dtype=np.float32)
b = cuda.pagelocked_empty(N, dtype=np.float32)
d = cuda.pagelocked_empty(N, dtype=np.float32)
c = cuda.pagelocked_empty(N, dtype=np.float32)

x = np.linspace(1, 100, N).astype(np.float32)
a[:] = np.sin(x)
b[:] = np.log(x)
d[:] = np.exp(-x / 50)

## Launch CUDA Kernel with Asynchronous Transfers

In [5]:
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
d_gpu = cuda.mem_alloc(d.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

stream = cuda.Stream()
start_event = cuda.Event()
end_event = cuda.Event()
start_event.record(stream)

cuda.memcpy_htod_async(a_gpu, a, stream)
cuda.memcpy_htod_async(b_gpu, b, stream)
cuda.memcpy_htod_async(d_gpu, d, stream)

fused_op(a_gpu, b_gpu, d_gpu, c_gpu, np.int32(N),
         block=(threads_per_block, 1, 1),
         grid=(blocks_per_grid, 1, 1),
         stream=stream)

cuda.memcpy_dtoh_async(c, c_gpu, stream)
end_event.record(stream)
end_event.synchronize()

kernel_time = start_event.time_till(end_event) * 1e-3

## Validate GPU Results and Report Timing

In [6]:
# CPU reference
start = time.time()
c_cpu = a * b + d
end = time.time()
cpu_time = end - start

# Compare with both relative and absolute tolerance
if np.allclose(c, c_cpu, rtol=1e-4, atol=1e-6):
    print("✅ Results match within tolerance.")
else:
    print("❌ Results differ. Showing mismatched values:")
    diffs = np.abs(c - c_cpu)
    idx = np.where(diffs > 1e-4)[0]
    for i in idx[:10]:  # Print first 10 differences
        print(f"Index {i}: GPU={c[i]:.6f}, CPU={c_cpu[i]:.6f}, Δ={diffs[i]:.2e}")
    print(f"Total mismatches: {len(idx)}")

# Print timings
print(f"NumPy FMA took: {cpu_time:.6f} seconds")
print(f"GPU (FMA kernel + overlap) time: {kernel_time:.6f} seconds")
print(f"🚀 Speedup: {cpu_time / kernel_time:.2f}×")

✅ Results match within tolerance.
NumPy FMA took: 0.030167 seconds
GPU (FMA kernel + overlap) time: 0.016037 seconds
🚀 Speedup: 1.88×
